In [1]:
import pandas as pd
import numpy as np

In [2]:
def load_data(file_path, format = 'txt'):
    if format == 'excel':
        df = pd.read_excel(file_path)
    elif format == 'csv':
        df = pd.read_csv(file_path)
    elif format == 'tsv':
        df = pd.read_csv(file_path, sep = '\t')
    elif format == 'txt':
        df = pd.read_table(file_path)
    else:
        raise ValueError('Invalid file format. The readable formats are "excel", "csv", "tsv" and "txt".')
    return df

def preprocess_data(df, cols):
    if df.isnull().values.any():
        df.dropna(inplace = True)
        df.reset_index(inplace = True)

    return df[cols]

def drop_small_orders(df, min_order_size = 5):
    return df[df.groupby('customer_ID').customer_ID.transform(len) >= min_order_size]

def expand_item_column(df, col):
    df_ohe = []
    df_ohe.append(df)
    df_ohe.append(pd.get_dummies(df[col], prefix = None, sparse = False))
    df = pd.concat(df_ohe, axis = 1)
    return df

def drop_cols(df, cols=[]):
    df = df.drop(cols, axis = 1)
    return df

In [3]:
file_path = 'data/All Transations - 2 Weeks.txt'
min_order_size = 20
customer_ID_col='customer_ID'

In [4]:
df = load_data(file_path, format = 'tsv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2107537 entries, 0 to 2107536
Data columns (total 6 columns):
customer_ID    int64
L1             object
L2             object
L3             object
sku            int64
brand          int64
dtypes: int64(3), object(3)
memory usage: 96.5+ MB


In [6]:
df.head(10)

,customer_ID,L1,L2,L3,sku,brand
0,168266,Power Tools,Power Saws and Accessories,Reciprocating Saw Blades,265105,2768
1,123986,Safety,Spill Control Supplies,Temporary Leak Repair,215839,586
2,158978,Hardware,Door Hardware,Thresholds,284756,1793
3,449035,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,12579,1231
4,781232,Motors,General Purpose AC Motors,General Purpose AC Motors,194681,2603
5,116599,Pneumatics,Pneumatic Tube Fittings,Pneumatic Push to Connect Tube Fittings,167757,3889
6,701116,Motors,General Purpose AC Motors,General Purpose AC Motors,310296,1068
7,555497,Motors,Motor Supplies,Capacitors,306732,1068
8,282317,Safety,Footwear and Footwear Accessories,Insoles,148549,2696
9,644437,Hand Tools,Sockets and Bits,Crowfoot Socket Wrenches,283869,3356


In [7]:
num_cols = list(df.select_dtypes(include=[np.number]).columns)
string_cols = list(df.select_dtypes(include=['O']).columns)

cols = string_cols + num_cols

print(cols)

['L1', 'L2', 'L3', 'customer_ID', 'sku', 'brand']


In [8]:
df = preprocess_data(df, cols)

In [9]:
df = drop_small_orders(df, min_order_size)

In [10]:
df = expand_item_column(df, ['L3'])

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 248099 entries, 58 to 2107483
Columns: 3795 entries, L1 to L3_pH Meters
dtypes: int64(3), object(3), uint8(3789)
memory usage: 909.7+ MB


In [12]:
dropping_cols = [x for x in cols if x != customer_ID_col]
print(dropping_cols)

['L1', 'L2', 'L3', 'sku', 'brand']


In [13]:
df = drop_cols(df, dropping_cols)

In [14]:
df.head(10)

,customer_ID,L3_12 Volt Accessories,L3_12-Point Flange Head Cap Screws,L3_3-Ring Binder Accessories,L3_3-Ring Binders,L3_3.3 Inch Diameter Motors,L3_4.4 Inch Diameter Motors,L3_5 X 20mm Glass and Ceramic Fuses,L3_5S Red Tag Stations,L3_A/C Mounting Pads,...,L3_Worker Emergency Identification,L3_Worm Gear Clamps,L3_Wrap-a-Round Tape Measures,L3_Wrist Rests and Palm Supports,L3_Wrist Supports and Wraps,L3_Y Strainers,L3_Yard Hydrants,L3_Zone Valve Actuators,L3_Zone Valves,L3_pH Meters
58,482732,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100,482732,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
142,482732,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
184,482732,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
226,482732,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
268,482732,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
310,482732,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
352,482732,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
372,838326,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
394,482732,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# df after: drop_small_orders , expand_item_column , drop_cols
df1 = df[:]
df2 = df[:]

In [16]:
df1 = df1.groupby(customer_ID_col).sum().reset_index()

In [17]:
data_cols = list(df.columns)
data_cols.remove('customer_ID')

In [18]:
df2 = df2.groupby(customer_ID_col).sum()[data_cols].reset_index()

In [19]:
df1.equals(df2)

True

In [20]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7453 entries, 0 to 7452
Columns: 3790 entries, customer_ID to L3_pH Meters
dtypes: int64(1), uint8(3789)
memory usage: 27.0 MB


In [21]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7453 entries, 0 to 7452
Columns: 3790 entries, customer_ID to L3_pH Meters
dtypes: int64(1), uint8(3789)
memory usage: 27.0 MB


In [22]:
df1.head(10)

,customer_ID,L3_12 Volt Accessories,L3_12-Point Flange Head Cap Screws,L3_3-Ring Binder Accessories,L3_3-Ring Binders,L3_3.3 Inch Diameter Motors,L3_4.4 Inch Diameter Motors,L3_5 X 20mm Glass and Ceramic Fuses,L3_5S Red Tag Stations,L3_A/C Mounting Pads,...,L3_Worker Emergency Identification,L3_Worm Gear Clamps,L3_Wrap-a-Round Tape Measures,L3_Wrist Rests and Palm Supports,L3_Wrist Supports and Wraps,L3_Y Strainers,L3_Yard Hydrants,L3_Zone Valve Actuators,L3_Zone Valves,L3_pH Meters
0,66334,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,66361,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
2,66619,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,66768,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,66849,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,66883,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,66916,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,67077,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
8,67226,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,67250,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df2.head(10)

,customer_ID,L3_12 Volt Accessories,L3_12-Point Flange Head Cap Screws,L3_3-Ring Binder Accessories,L3_3-Ring Binders,L3_3.3 Inch Diameter Motors,L3_4.4 Inch Diameter Motors,L3_5 X 20mm Glass and Ceramic Fuses,L3_5S Red Tag Stations,L3_A/C Mounting Pads,...,L3_Worker Emergency Identification,L3_Worm Gear Clamps,L3_Wrap-a-Round Tape Measures,L3_Wrist Rests and Palm Supports,L3_Wrist Supports and Wraps,L3_Y Strainers,L3_Yard Hydrants,L3_Zone Valve Actuators,L3_Zone Valves,L3_pH Meters
0,66334,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,66361,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
2,66619,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,66768,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,66849,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,66883,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,66916,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,67077,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
8,67226,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,67250,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
